In [1]:
import pandas as pd
import numpy as np

# Loading Raw Data

In [38]:
# os.path.join(“..“, “raw_data”, “companies.csv”)
comps = pd.read_csv('..\\raw_data\\companies.csv')
acquisitions = pd.read_csv('..\\raw_data\\acquisitions.csv')
degrees = pd.read_csv('..\\raw_data\\degrees.csv')
founders = pd.read_csv('..\\raw_data\\founders.csv')
rounds = pd.read_csv('..\\raw_data\\funding-rounds.csv')
funds = pd.read_csv('..\\raw_data\\funds.csv')
investments = pd.read_csv('..\\raw_data\\investments.csv')
ipos = pd.read_csv('..\\raw_data\\ipos.csv')
milestones = pd.read_csv('..\\raw_data\\milestones.csv')
people = pd.read_csv('..\\raw_data\\people.csv')
relationships =pd.read_csv('..\\raw_data\\relationships.csv')

# Feature Engineering

#### Change to use in all the functions

In [3]:
comps = comps.rename(columns={'id': 'object_id'})

## 2  1 - Time between founding date and specific investment (Diff_time_founded_invest)

changes are made !!!!

In [60]:
def diff_foundation_fundround(comps_df, rounds_df):
    
    '''Returns a merge table between companies and funding rounds. One column added: difference between the date of the companies fundations
    and each round of investment , in MONTHS. It includes companies_id repeated for each round that is realated with the company '''
    
    comps = comps_df
    rounds = rounds_df
    
    comps = comps.rename(columns={'id': 'object_id'})
    comps_rounds = comps.merge(rounds, how = 'left', on = 'object_id')
    comps_rounds.founded_at = pd.to_datetime(comps_rounds.founded_at)
    comps_rounds.funded_at = pd.to_datetime(comps_rounds.funded_at)
    comps_rounds['Diff_time_founded_invest'] = abs(comps_rounds.funded_at - comps_rounds.founded_at)/np.timedelta64(12, 'M')
    
    return comps_rounds

## 3  3 - Time between Series A and specific investment

#changes are made !!!!!!!!!

In [28]:
def time_serie_investments(rounds_data, reference):

    ''' company id partitioned. Take a serie X as reference: values = (a, b, c, angel, any on funding_round_code). And create a new column 
    'data_serie_reference' which contains the min date of this serie for each company.
    And also create a new column 'time_serie_investment' with the difference between each round and the date of reference serie, in MONTHS'''
    
    rounds = rounds_data
    rounds.funded_at = pd.to_datetime(rounds.funded_at)
    min_dates = {}
    for id_company in rounds[rounds.funding_round_code == reference].object_id.unique():
        #list_dates_by_comp: list of each company which has a serie X
        list_dates_by_comp = rounds[rounds.funding_round_code == reference].funded_at[rounds.object_id == id_company]
        min_dates[id_company] = min(list_dates_by_comp)

    rounds['date_serie_reference'] = rounds.object_id.map(min_dates)
    rounds['time_serie_investment'] =  (rounds.funded_at - rounds.date_serie_reference)/np.timedelta64(12, 'M')
    
    return rounds

## [Dependent on 1] 2 - Time to series A round 

In [37]:
def diff_fund_found_filter(comps, rounds, serie_filtered):
    ''' filter the table from diff_foundation_foundround (merge comps and rounds), by a particular serie (values = a, b ,c ,angel, venture?)'''
    get_table = diff_foundation_fundround(comps, rounds)
    
    return get_table[get_table.funding_round_code == serie_filtered]

## [Dependent on 3] 4 - Number of rounds before Series A

In [85]:
def num_rounds_before(comps, rounds, serie_reference):
    ''' return the companies table with a new column num_round_before which is the number of rounds before the serie you want. time_serie_investments applied inside.'''

    comps = comps.rename(columns={'id': 'object_id'})
    get_table = time_serie_investments(rounds, serie_reference)
    get_table = get_table[get_table.time_serie_investment<0]
    get_table = get_table.groupby('object_id').count()
    get_table = get_table['id'].reset_index(name="num_round_before")
    merge = comps.merge(get_table, how ='left', on = 'object_id')
    return merge

## 7 - Time between Series A and IPO

Make the function in a way that we can easily replace Series A with something else if we decide to switch

    Take date of series A round from funding rounds table (keep only first if several)
    Take date of IPO from IPO table
    Do a left merge on comp ID from IPO to funding rounds
    Create column = date IPO - date Series A
    merge that with company table



In [53]:
#to be merged to companies table
def date_ipo_serie(rounds, ipos, serie_reference):
    ''' get a table '''
    get_rounds = time_serie_investments(rounds, serie_reference)
    get_rounds = get_rounds[['object_id', 'date_serie_reference']].drop_duplicates()
    
    merge = ipos.merge(get_rounds, how = 'left', on = 'object_id')

    merge.public_at = pd.to_datetime(merge.public_at)
    merge['data_serie_reference'] = pd.to_datetime(merge['date_serie_reference'])
    
    merge['diff_ipo_serie'] = (merge['public_at'] - merge['date_serie_reference'])/np.timedelta64(12, 'M')
    
    return merge.T.drop_duplicates().T

In [54]:
df =date_ipo_serie(rounds, ipos, 'angel')

In [37]:
rounds[rounds.object_id=='c:59']

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at,date_serie_reference,time_serie_investment
357,380,380,c:59,1998-08-01,angel,angel,100000.0,100000.0,USD,0.0,...,1,0,1,http://www.google.com/corporate/history.html,NaN,initial-importer,2007-08-18 09:03:04,2011-08-01 20:45:59,1998-08-01,0.000000
358,381,381,c:59,1999-06-07,series-a,a,25000000.0,25000000.0,USD,0.0,...,2,1,0,http://www.google.com/corporate/history.html,NaN,initial-importer,2007-08-18 09:04:02,2011-08-01 20:45:59,1998-08-01,0.848751


In [57]:
rounds[rounds.object_id == 'c:2720']

,id,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,raised_amount,raised_currency_code,pre_money_valuation_usd,...,participants,is_first_round,is_last_round,source_url,source_description,created_by,created_at,updated_at,date_serie_reference,time_serie_investment
1532,1660,1660,c:2720,1996-01-01,series-a,a,1600000.0,1600000.0,USD,0.0,...,1,0,1,http://www.xogroupinc.com/the-knot-inc-company...,The Knot Milestones,initial-importer,2008-02-29 00:38:24,2011-08-22 02:19:36,NaT,NaN
1533,1661,1661,c:2720,1999-04-15,series-c+,c,15000000.0,15000000.0,USD,0.0,...,1,1,0,http://findarticles.com/p/articles/mi_m0EIN/is...,The Knot Receives Strategic Investment From QVC,initial-importer,2008-02-29 00:38:25,2008-02-29 00:38:25,NaT,NaN
1534,1662,1662,c:2720,1998-04-01,series-b,b,3000000.0,3000000.0,USD,0.0,...,1,0,0,http://www.xogroupinc.com/the-knot-inc-company...,The Knot Milestones,initial-importer,2008-02-29 00:38:25,2011-08-22 02:19:36,NaT,NaN


In [55]:
df

,ipo_id,object_id,valuation_amount,valuation_currency_code,stock_symbol,source_url,source_description,public_at,date_serie_reference,diff_ipo_serie
0,1,c:1654,NaN,USD,NASDAQ:AAPL,NaN,NaN,1980-12-19 00:00:00,NaT,NaN
1,2,c:1242,NaN,USD,NASDAQ:MSFT,NaN,NaN,1986-03-13 00:00:00,NaT,NaN
2,3,c:342,NaN,USD,NYSE:DIS,NaN,NaN,1969-06-09 00:00:00,NaT,NaN
3,4,c:59,NaN,USD,NASDAQ:GOOG,NaN,NaN,2004-08-25 00:00:00,1998-08-01 00:00:00,6.0672
4,5,c:317,1e+11,USD,NASDAQ:AMZN,NaN,NaN,1997-05-01 00:00:00,NaT,NaN
5,6,c:1438,3.5e+08,USD,SFLY,http://www.nasdaq.com/markets/ipos/company/shu...,NaN,2006-09-29 00:00:00,NaT,NaN
6,7,c:2644,1.6e+08,USD,DIVX,NaN,NaN,2006-05-01 00:00:00,NaT,NaN
7,8,c:2584,NaN,USD,OPESF,NaN,NaN,2004-03-01 00:00:00,NaT,NaN
8,9,c:2720,NaN,USD,XOXO,NaN,NaN,1999-12-02 00:00:00,NaT,NaN
9,11,c:3316,6e+09,USD,NASDAQ:BMC,NaN,NaN,1988-08-12 00:00:00,NaT,NaN
